# Device Environment Check

In [1]:
import torch

print(f"cuda activate : {torch.cuda.is_available()}\n") #gpu
print(f"python_version : {sys.version}\n") #python
print(f"torch_version : {torch.__version__}\n") #pytorch
print(f"cuda_version : {torch.cuda_version}\n") #cuda
print(f"cudnn_version : {torch.backends.cudnn.version()}\n") #cudnn

c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda activate : True

python_version : 3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]

torch_version : 1.12.1

cuda_version : 11.6

cudnn_version : 8302



In [2]:
import os
import sys
from datetime import datetime

os.getcwd() #디렉토리 확인
#sys.path.append('') #필요 lib나 링크 추가할 때 쓰는 코드

'c:\\Users\\Administrator\\Desktop\\study\\lecture\\Part 5\\Untitled Folder'

# Data Preprocessing

In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch import utils
from torch.utils.data import DataLoader
from torch.optim import RAdam

import wandb
import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST 

In [11]:
#download data
#prepare to download data
#NLP data는 다름
data_root = os.path.join(os.getcwd(), "data")

transform = torchvision.transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5],[0.5]),
    ]
)
fashion_mnist_dataset = FashionMNIST(data_root, download=True, transform=transform)

In [12]:
"""Loads the data"""
# Create indices for the split
batch_size=100

dataset_size = len(fashion_mnist_dataset)
val_size = int(0.1 * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = torch.utils.data.random_split(fashion_mnist_dataset,
                                               [train_size, val_size])

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True)
val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=True)


In [6]:
# Waarmup Scheduler

class WarmupLR(optim.lr_scheduler.LambdaLR) :
    def __init__ (
        self, 
        optimizer:optim.Optimizer,
        warmup_end_steps: int,
        last_epoch : int = -1,    
        ):

        def warmup_fn(step: int):
            if step < warmup_end_steps:
                return float(step) / float(max(warmup_end_steps,1))
            return 1.0

        super().__init__(optimizer, warmup_fn, last_epoch)

# Model Design

In [7]:
# With some modifications, source is from https://github.com/Bjarten/early-stopping-pytorch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.ckpt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.ckpt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        
        filename = self.path.split('/')[-1]
        save_dir = os.path.dirname(self.path)
        print(save_dir, f"val_loss-{val_loss}-{filename}")
        torch.save(model, f = os.path.join(save_dir, f"val_loss-{val_loss}"))
        self.val_loss_min = val_loss

In [8]:
class LeNet(nn.Module):
	def __init__(self, numChannels, classes):
		super(LeNet, self).__init__()
		#Layer1
		self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=20,kernel_size=(5, 5))
		self.relu1 = nn.ReLU()
		self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		#Layer2
		self.conv2 = nn.Conv2d(in_channels=20, out_channels=50,kernel_size=(5, 5))
		self.relu2 = nn.ReLU()
		self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		self.fc1 = nn.Linear(in_features=800, out_features=500)
		self.relu3 = nn.ReLU()
		self.fc2 = nn.Linear(in_features=500, out_features=classes)
		self.logSoftmax = nn.LogSoftmax(dim=1)
    
	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv1(x)
		x = self.relu1(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu2(x)
		x = self.maxpool2(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = torch.flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output

## Define Model

In [9]:
#define model.
# crtl + slash => 주석 한번에 달기
#model = MLP(28*28, 128, 64, 10)
#model = MLPWithDropout(28*28, 256,128, 64, 10, dropout_prob=0.3)

model = LeNet(1,len(fashion_mnist_dataset.classes))
model_name = type(model).__name__

#define loss
loss_function = nn.CrossEntropyLoss()

#define optimizer
lr = 1e-3
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = RAdam(model.parameters(), lr=lr)
optimizer_name = type(optimizer).__name__

#define scheduler
scheduler = WarmupLR(optimizer, 1500)
scheduler_name = type(scheduler).__name__ if scheduler is not None else "No"

max_epoch = 100

#define tensorboard logger
#log_dir = os.path.join("runs", model_name)

now = datetime.now()
run_name = f"{now.strftime('%Y-%m-%d %H%M')}-{model_name}-{optimizer_name}_optim_{lr}_lr_with_{scheduler_name}_scheduler" 
log_dir = f"runs\{now.strftime('%Y-%m-%d %H%M')}-{model_name}"

log_interval = 100

#define wandb

project_name = "fashion_mnist_tutorials"
run_tags = [project_name]
wandb.init(
    project = project_name,
    name= run_name,
    tags= run_tags,
    config={"lr" : lr, "model_name" : model_name, "optimizer_name" : optimizer_name , "scheduler_name" : scheduler_name},
    reinit=True

)

#save model path+
log_model_path = os.path.join(log_dir, "models")
os.makedirs(log_model_path, exist_ok=True)
print(log_model_path)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: magicturtle. Use `wandb login --relogin` to force relogin


runs\2022-09-25 1539-LeNet\models


In [13]:
#define early stopping
early_stopper = EarlyStopping(
    patience=3, verbose=True, path= os.path.join(log_model_path,"model.ckpt")
)

train_step = 0
for epoch in range(1, max_epoch+1):
    #validation step
    #validation 할 때 optimizer가 train에 관여하지 않도록 해야 함
    with torch.no_grad():
        val_loss = 0.0
        val_corrects =0
        model.eval()
        
        for val_batch_idx, (val_images, val_labels) in enumerate (
            tqdm(val_loader, position=0, leave=True, desc = "validation") #중간중간 output을 예쁘게 보이게
        ):
            # forwards
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs, 1)

            # loss & acc
            val_loss += loss_function(val_outputs, val_labels) / val_outputs.shape[0] #val_outputs.shape는 batchsize
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0]
     
    #vaild step logging
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_acc = val_corrects / len(val_loader)

    print(
        f"{epoch} epoch, {train_step} step: val_loss : {val_epoch_loss}, val_acc: {val_epoch_acc}"
        )
    
    #wandb log
    wandb.log({
        "Loss/val": val_epoch_loss,
        "Acc/val" : val_epoch_acc,
        "Images/val" : wandb.Image(val_images),
        "Preds/val" : wandb.Histogram(val_outputs.detach().numpy()),
        "Outputs/val" : wandb.Histogram(val_preds.detach().numpy()),
        "Labels/val" : wandb.Histogram(val_labels.data.detach().numpy()),
        }
        , step=train_step
    )

    #check model early stopping poing & save model if model reached the best performance.
    early_stopper(val_epoch_loss, model)
    if early_stopper.early_stop :
        break

    current_loss = 0
    current_correct= 0 
    model.train()


    #train step    
    for batch_idx, (images, labels) in enumerate(
        tqdm(train_loader, position=0, leave=True, desc = "train") #중간중간 output을 예쁘게 보이게
    ):
        
        current_loss = 0.0
        current_corrects = 0
        
        #forwards
        #get predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
      
        #get loss (Loss 계산)
        loss = loss_function(outputs, labels)
        
        #backpropagation
        #opmizier 초기화
        optimizer.zero_grad()
        
        # Perform backward pass
        loss.backward()
        
        # Perform Optimization
        optimizer.step()

        # Perform Scheduler
        if scheduler is not None :
           scheduler.step()
        
        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)

        if train_step % log_interval == 0:
           train_loss = current_loss / log_interval
           train_acc = current_corrects / log_interval
            
           print(
                    f"{train_step} : train_loss : {train_loss}, train_acc: {train_acc}"
            )

           # wandb log 
           wandb.log({
           "Loss/train": train_loss,
           "Acc/train" : train_acc,
           "Images/train" : wandb.Image(val_images),
           "Preds/train" : wandb.Histogram(outputs.detach().numpy()),
           "Outputs/train" : wandb.Histogram(preds.detach().numpy()),
           "Labels/train" : wandb.Histogram(labels.data.detach().numpy()),
           "Leanring Rate" : scheduler.get_last_lr()[0]
           }
            , step=train_step
           )
           current_loss = 0
           current_correct= 0 
            
        train_step +=1
        


validation: 100%|██████████| 60/60 [00:01<00:00, 31.08it/s]


1 epoch, 0 step: val_loss : 0.023053469136357307, val_acc: 0.10233331471681595
Validation loss decreased (inf --> 0.023053).  Saving model ...
runs\2022-09-25 1539-LeNet\models val_loss-0.023053469136357307-runs\2022-09-25 1539-LeNet\models\model.ckpt


train:   0%|          | 1/540 [00:00<00:56,  9.61it/s]

0 : train_loss : 0.023037714958190916, train_acc: 0.07000000029802322


train:  19%|█▉        | 103/540 [00:06<00:27, 15.75it/s]

...

6300 : train_loss : 0.0012345672398805618, train_acc: 0.9599999785423279


train:  86%|████████▌ | 462/540 [00:25<00:04, 16.91it/s]

6400 : train_loss : 0.001049957573413849, train_acc: 0.949999988079071


validation: 100%|██████████| 60/60 [00:01<00:00, 32.01it/s]


13 epoch, 6480 step: val_loss : 0.002735863672569394, val_acc: 0.9144998788833618
EarlyStopping counter: 3 out of 3


In [14]:
#save model
os.makedirs("./logs/models", exist_ok=True)
torch.save(model, os.path.join(log_model_path, "model.ckpt"))

In [16]:
#load model
loaded_model = torch.load(os.path.join(log_model_path,"val_loss-0.0025349785573780537"))
loaded_model.eval()
print(loaded_model)

LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (logSoftmax): LogSoftmax(dim=1)
)


In [17]:
def softmax(x, axis=0):
    "numpy softmax"
    max = np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x-max)
    sum = np.sum(e_x, axis=axis, keepdims=True)
    f_x = e_x / sum
    return f_x

In [18]:
#test dataset download
test_batch_size = 100
test_dataset = FashionMNIST(data_root, download=True, train=False, transform=transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size= test_batch_size, shuffle=False, num_workers=1)


test_labels_list = []
test_preds_list = []
test_outputs_list = []
for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc = "testing")):
    #forward
    test_outputs = loaded_model(test_images)
    _, test_preds = torch.max(test_outputs,1)

    final_outs = softmax(test_outputs.detach().numpy(), axis=1)
    test_outputs_list.extend(final_outs)
    test_preds_list.extend(test_preds.detach().numpy()) #gpu로 연산한 데이터는 넘파이로 사용할 때 반드시 detach로 풀어써줘서 cpu로 옮겨줘야함
    test_labels_list.extend(test_labels.detach().numpy()) #gpu로 연산한 데이터는 넘파이로 사용할 때 반드시 detach로 풀어써줘서 cpu로 옮겨줘야함
    

test_preds_list = np.array(test_preds_list) #acc를 구하기 위해 numpy로 바꿔
test_labels_list = np.array(test_labels_list)

print(f"acc: {np.mean(test_preds_list == test_labels_list)*100}")



testing: 100%|██████████| 100/100 [00:03<00:00, 29.47it/s]

acc: 84.13000000000001
